In [ ]:
import configparser
import psycopg2
from time import time
from iac import create_cluster, delete_cluster
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%load_ext sql

In [ ]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

DWH_DB= config.get("CLUSTER","DB_NAME")
DWH_DB_USER= config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD= config.get("CLUSTER","DB_PASSWORD")
DWH_PORT = config.get("CLUSTER","DB_PORT")

host = create_cluster()
conn_string = "postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, host, DWH_PORT, DWH_DB)

%sql $conn_string

In [ ]:
%%sql
CREATE SCHEMA IF NOT EXISTS nodist;
SET search_path TO nodist;

CREATE TABLE users(
    user_id    varchar    PRIMARY KEY, 
    first_name varchar    NOT NULL, 
    last_name  varchar    NOT NULL, 
    gender     varchar(1) NOT NULL, 
    level      varchar    NOT NULL);

CREATE TABLE artists(
    artist_id varchar PRIMARY KEY, 
    name      varchar NOT NULL, 
    location  varchar, 
    latitude  decimal, 
    longitude decimal);

CREATE TABLE songs(
    song_id   varchar  PRIMARY KEY, 
    title     varchar  NOT NULL, 
    artist_id varchar  NOT NULL REFERENCES artists, 
    year      smallint NOT NULL, 
    duration  decimal  NOT NULL);

CREATE TABLE time(
    start_time timestamp PRIMARY KEY, 
    hour       smallint, 
    day        smallint, 
    weekofyear smallint, 
    month      smallint, 
    year       smallint, 
    weekday    smallint);

CREATE TABLE songplays(
    songplay_id int       IDENTITY(0,1)       PRIMARY KEY, 
    start_time  timestamp REFERENCES time   , 
    user_id     int       REFERENCES users, 
    level       varchar   NOT NULL, 
    song_id     varchar   REFERENCES songs  , 
    artist_id   varchar   REFERENCES artists, 
    session_id  int       NOT NULL, 
    location    varchar, 
    user_agent  varchar);

In [ ]:
%%sql

INSERT INTO nodist.users
SELECT t.userId, t.firstName, t.lastName, t.gender, se.level
FROM dist.staging_events se
JOIN
    (SELECT userId, firstName, lastName, gender, MAX(ts) max_ts
    FROM dist.staging_events
    WHERE page='NextSong'
    GROUP BY userId, firstName, lastName, gender) t
ON se.userId = t.userId and se.firstName = t.firstName and se.lastName = t.lastName 
AND se.gender = t.gender and se.ts = t.max_Ts;

INSERT INTO nodist.songs
SELECT DISTINCT song_id, title, artist_id, year, duration
FROM dist.staging_songs;

INSERT INTO nodist.artists
SELECT t.artist_id, ss.artist_name, ss.artist_location, ss.artist_latitude
FROM dist.staging_songs ss
JOIN
    (SELECT artist_id, MIN(song_id) song_id
    FROM dist.staging_songs
    GROUP BY artist_id) t
ON ss.artist_id = t.artist_id
AND ss.song_id = t.song_id;

INSERT INTO nodist.time
SELECT DISTINCT TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second' start_time,
    EXTRACT(hour from start_time),
    EXTRACT(day from start_time),
    EXTRACT(w from start_time),
    EXTRACT(month from start_time),
    EXTRACT(year from start_time),
    EXTRACT(dow from start_time)
FROM dist.staging_events
WHERE page='NextSong';

INSERT INTO nodist.songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT DISTINCT TIMESTAMP 'epoch' + se.ts/1000 *INTERVAL '1 second' start_time,
    se.userId, se.level, s.song_id, s.artist_id, se.sessionId, se.location, se.userAgent
FROM dist.staging_events se
JOIN artists a ON a.name = se.artist
JOIN songs s ON s.title = se.song and s.duration = se.length
WHERE se.page='NextSong'

In [ ]:
%%sql
SELECT table_id,"table" tablename,schema schemaname,tbl_rows,unsorted,sortkey1,sortkey_num,diststyle 
FROM svv_table_info
ORDER BY schemaname, tablename

#### 1. Display a play list of the specific user in the latest played order.
<table>
    <tr>
        <th>title</th>
        <th>artist</th>
        <th>play_date</th>
    </tr>
    <tr>
        <td>Rianna</td>
        <td>Fisher</td>
        <td>2018-11-28</td>
    </tr>
    <tr>
        <td>I CAN&#x27;T GET STARTED</td>
        <td>Ron Carter</td>
        <td>2018-11-27</td>
    </tr>
    <tr>
        <td>Shimmy Shimmy Quarter Turn (Take It Back To Square One)</td>
        <td>Hellogoodbye</td>
        <td>2018-11-26</td>
    </tr>
    <tr>
        <td>Emergency (Album Version)</td>
        <td>Paramore</td>
        <td>2018-11-26</td>
    </tr>
    <tr>
        <td>What It Ain&#x27;t</td>
        <td>Josh Turner</td>
        <td>2018-11-26</td>
    </tr>
    <tr>
        <td>Eye Of The Beholder</td>
        <td>Metallica</td>
        <td>2018-11-26</td>
    </tr>
    <tr>
        <td>Loneliness</td>
        <td>Tomcraft</td>
        <td>2018-11-24</td>
    </tr>
    <tr>
        <td>Bang! Bang!</td>
        <td>The Knux</td>
        <td>2018-11-24</td>
    </tr>
    <tr>
        <td>You&#x27;re The One</td>
        <td>Dwight Yoakam</td>
        <td>2018-11-24</td>
    </tr>
    <tr>
        <td>Sun / C79</td>
        <td>Cat Stevens</td>
        <td>2018-11-24</td>
    </tr>
    <tr>
        <td>Wax on Tha Belt (Baby G Gets Biz)</td>
        <td>Mad Flava</td>
        <td>2018-11-24</td>
    </tr>
    <tr>
        <td>Catch You Baby (Steve Pitron &amp; Max Sanna Radio Edit)</td>
        <td>Lonnie Gordon</td>
        <td>2018-11-23</td>
    </tr>
    <tr>
        <td>Nothin&#x27; On You [feat. Bruno Mars] (Album Version)</td>
        <td>B.o.B</td>
        <td>2018-11-21</td>
    </tr>
    <tr>
        <td>Die Kunst der Fuge_ BWV 1080 (2007 Digital Remaster): Contrapunctus XVII - Inversus</td>
        <td>Lionel Rogg</td>
        <td>2018-11-21</td>
    </tr>
    <tr>
        <td>Mr. Jones</td>
        <td>Counting Crows</td>
        <td>2018-11-21</td>
    </tr>
    <tr>
        <td>You&#x27;re The One</td>
        <td>Dwight Yoakam</td>
        <td>2018-11-09</td>
    </tr>
</table>

In [ ]:
queries = []

queries.append("""
SET enable_result_cache_for_session TO OFF;
SET search_path TO {};

SELECT title, name artist, TO_CHAR(start_time, 'YYYY-MM-DD') play_date
FROM songs s
JOIN songplays sp ON s.song_id=sp.song_id
JOIN artists a ON sp.artist_id=a.artist_id
WHERE user_id=88
ORDER BY start_time DESC
""")

#### 2. What is the most played song every year?
<table>
    <tr>
        <th>year</th>
        <th>title</th>
        <th>artist</th>
        <th>play_count</th>
    </tr>
    <tr>
        <td>2018</td>
        <td>You&#x27;re The One</td>
        <td>Dwight Yoakam</td>
        <td>37</td>
    </tr>
</table>

In [ ]:
queries.append("""
SET enable_result_cache_for_session TO OFF;
SET search_path TO {};

WITH t1 as (SELECT t.year, title, name, count(songplay_id)
FROM songplays sp 
JOIN songs s ON s.song_id=sp.song_id
JOIN artists a ON a.artist_id=sp.artist_id
JOIN time t ON sp.start_time=t.start_time
GROUP BY t.year, title, name
ORDER BY 3 DESC),

t2 as (SELECT year, max(count)
FROM t1
GROUP BY year)

SELECT t1.year, t1.title, t1.name artist, t1.count play_count
FROM t1, t2
WHERE t1.year=t2.year and t1.count=t2.max""")

#### 3. Display the 5 most played artists from LA.
<table>
    <tr>
        <th>rank</th>
        <th>artist</th>
        <th>play_count</th>
    </tr>
    <tr>
        <td>1</td>
        <td>Linkin Park</td>
        <td>4</td>
    </tr>
    <tr>
        <td>1</td>
        <td>Metallica</td>
        <td>4</td>
    </tr>
    <tr>
        <td>3</td>
        <td>Black Eyed Peas</td>
        <td>3</td>
    </tr>
    <tr>
        <td>4</td>
        <td>Katy Perry</td>
        <td>1</td>
    </tr>
    <tr>
        <td>4</td>
        <td>Maroon 5</td>
        <td>1</td>
    </tr>
</table>

In [ ]:
queries.append("""
SET enable_result_cache_for_session TO OFF;
SET search_path TO {};

SELECT rank() over (order by play_count DESC), a.name artist, count(songplay_id) play_count
FROM songplays sp
JOIN artists a ON a.artist_id=sp.artist_id
WHERE a.location = 'Los Angeles, CA'
GROUP BY 2
ORDER BY 1
LIMIT 5
""")

#### 4. What time are women most likely to listen music?
<table>
    <tr>
        <th>hour</th>
        <th>play_count</th>
    </tr>
    <tr>
        <td>17</td>
        <td>27</td>
    </tr>
    <tr>
        <td>15</td>
        <td>18</td>
    </tr>
    <tr>
        <td>18</td>
        <td>15</td>
    </tr>
    <tr>
        <td>16</td>
        <td>14</td>
    </tr>
    <tr>
        <td>14</td>
        <td>13</td>
    </tr>
    <tr>
        <td>11</td>
        <td>13</td>
    </tr>
    <tr>
        <td>8</td>
        <td>12</td>
    </tr>
    <tr>
        <td>20</td>
        <td>12</td>
    </tr>
    <tr>
        <td>19</td>
        <td>10</td>
    </tr>
    <tr>
        <td>21</td>
        <td>10</td>
    </tr>
</table>

In [ ]:
queries.append("""
SET enable_result_cache_for_session TO OFF;
SET search_path TO {};

SELECT t.hour, count(songplay_id) play_count
FROM songplays sp
JOIN users u ON u.user_id=sp.user_id
JOIN time t ON sp.start_time=t.start_time
WHERE u.gender = 'F'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")

In [ ]:
def compareQueryTimes(schemas):
    df = pd.DataFrame({"query":[1, 2, 3, 4]}).set_index("query")
    for schema in schemas:
        query_times = []
        for query in queries:
            query = query.format(schema)
            t0 = time()
            %sql $query
            dt = time() - t0
            query_times.append(dt)
        print(query_times)
        df["time_"+schema] = query_times
    return df
df = compareQueryTimes(["dist", "nodist"])

In [ ]:
df.plot.bar(title = "Query time with/without distkey")
plt.show()

In [ ]:
df["imp"] = 100 * (df["time_nodist"] - df["time_dist"]) / df["time_nodist"]

In [ ]:
df[["imp"]].plot.bar(title = "Improvement by distkey(%)")
plt.show()

In [ ]:
%%sql
DROP SCHEMA nodist CASCADE;

In [ ]:
delete_cluster()